# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import re
from datetime import datetime, date, time

In [2]:
data = pd.read_csv('main_task.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [3]:
"""Заменим значение колонки "Price Range" на числовые: NaN - 0, '$' - 1, '$$ - $$$' - 2, '$$$$' - 3"""
data['Price Range'] = data['Price Range'].apply(lambda x: 1 if x == '$' else (2 if x == '$$ - $$$' else (3 if x == '$$$$' else 0)))

In [4]:
"""По каждому типу кухни создадим отдельный признак, показывающий наличие в ресторане данного типа кухни"""
# Так как все рестораны расположены в городах Европы, то заменим пустое значение в колонке Cuisine Style на ['European']
data['Cuisine Style'].fillna("['European']", inplace=True)
cuisine_styles = set() #создаем пустое множество, куда сохраним все уникальные значения типов кухонь
    
for item in data['Cuisine Style'].apply(lambda x: str(x)[2:-2]):
    for cuisine_style in item.split("', '"):
        cuisine_styles.add(cuisine_style)
    
cuisine_styles.discard('') #удаляем пустое значение из полуившегося множества

# Создаем в dataframe колонки с названиями типов кухонь и заполняем 1 в случае, если такой тип кухни есть в ресторане
for item in cuisine_styles:
    data[item] = data['Cuisine Style'].apply(lambda x: 1 if item in str(x)[2:-2] else 0)

In [5]:
"""Заменим в колонке Cuisine Style значения на количество типов кухонь в данном ресторане, если информации нет, то запишем 1"""
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: len(str(x).split("', '")) if not pd.isna(x) else 1)

In [6]:
"""Создаем в датасэте колонки с названиями городов и заполняем 1 в случае, если ресторан находится в текущем городе"""
data = pd.get_dummies(data, columns=['City'], dummy_na=True)
data = data.drop(['City_nan'], axis = 1) #удалим признак, так как все его значения равны 0

In [7]:
"""Обработаем данные об отзывах. Разнесем отзывы на отдельные колонки датасета, заполним пропуски в Number of Reviews"""
# С помощью регулярных выражений достанем отзывы и поместим их в колнки review_1 и review_2
data['review_1'] = data['Reviews'].apply(lambda x: str(re.findall(r'\[\[\'[a-zA-Z0-9_ ,.!?()`´+-=*:;]*\'', x))[5:-3])
data['review_2'] = data['Reviews'].apply(lambda x: str(re.findall(r', \'[a-zA-Z0-9_ ,.!?()`´+-=*:;]*\'\],', x))[5:-5])

# Напишем функцию, которая заполнит пропуски в столбце Number of Reviews.
# Если нет отзывов - 0, если есть хотя бы один отзыв - 1, если есть два отзыва - 2
def number_of_reviews_fillna(df):
    if pd.isna(df['Number of Reviews']):
        if df['review_1'] == '':
            df['Number of Reviews'] = 0
        elif  df['review_2'] == '':
            df['Number of Reviews'] = 1
        else:
            df['Number of Reviews'] = 2
    return df

# Применим функцию к нашему датасету
data = data.apply(number_of_reviews_fillna, axis = 1)

In [8]:
"""Создим признак, показывающий разницу между датами отзывов в днях"""
# С помощью регулярных выражений достанем даты отзывов и поместим их в колнки review_date_1 и review_date_2
data['review_date_1'] = data['Reviews'].apply(lambda x: str(re.findall(r'\[\'\d\d/\d\d/\d\d\d\d\'', x))[4:-3])
data['review_date_2'] = data['Reviews'].apply(lambda x: str(re.findall(r'\'\d\d/\d\d/\d\d\d\d\'\]', x))[3:-4])
    
# Переведем полученные данные в datetime
data['review_date_1'] = data['review_date_1'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y')if x !='' else x) 
data['review_date_2'] = data['review_date_2'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y')if x !='' else x)

# Сгенерируем новый признак
data['review_date_delta'] = (data['review_date_1'] - data['review_date_2'])
data['review_date_delta'] = data['review_date_delta'].apply(lambda x: abs(int(x.days)) if not pd.isna(x) else 0)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 170 entries, Restaurant_id to review_date_delta
dtypes: datetime64[ns](2), float64(3), int64(159), object(6)
memory usage: 51.9+ MB


In [10]:
"""Удалим все нечисловые столбцы"""
data = data.drop(['Restaurant_id', 'Reviews', 'review_1', 'review_2', 'URL_TA', 'ID_TA', 'review_date_1', 'review_date_2'], axis = 1)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 162 entries, Cuisine Style to review_date_delta
dtypes: float64(3), int64(159)
memory usage: 49.4 MB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [12]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = data.drop(['Rating'], axis = 1)
y = data['Rating']

In [13]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [14]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [15]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [16]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [17]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21205349999999998
